In [2]:
pip install python-dotenv 'crewai[tools]' langchain-openai -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

True

In [1]:
from langchain_openai import ChatOpenAI
from crewai import Agent, Process, Crew, Task
from IPython.display import display, Markdown
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

In [3]:
llm = ChatOpenAI(model="gpt-5.2", temperature=0.5)

In [ ]:
interviewer = input("Enter the name of the interviewer: ")
# Matheus
company = input("Enter the name of the company: ")
# Eyna
job_position = input("Enter the job position: ")
# Desenvolvedor(a) Full Stack Júnior
job_description = input("Enter the job description: ")
# Criar e manter APIs e integrações para
# │  que clientes consigam consumir nossos modelos de visão computacional em escala (edge + nuvem). # Garantir que  │
# │  as soluções sejam performáticas, seguras, observáveis e fáceis de manter, seguindo boas práticas de            │
# │  engenharia e AWS. # Trabalhar lado a lado com o time de Machine Learning/Computer Vision, ajudando a levar     │
# │  modelos de IA para produção em ambientes reais. # Usar IA para aumentar sua produtividade (por exemplo, para   │
# │  gerar código, testes, documentação, debugging assistido, etc.). # Transmitir conhecimento para o time de       │
# │  forma simples, por meio de tutoriais, mini-guias, exemplos de código e pequenos vídeos ou gravações de tela    │
# │  (não precisa ser especialista em vídeo — o que importa é a disposição e o espírito de time).   # O que você    │
# │  vai fazer no dia a dia # Desenvolver e manter serviços back-end (REST/GraphQL) com boas práticas de código,    │
# │  logs e testes básicos. # Ajudar a implementar interfaces web (dashboards, telas de configuração,               │
# │  monitoramento) com foco em clareza para o usuário. # Apoiar na configuração e evolução de recursos em AWS      │
# │  (ex.: API Gateway/ALB, Lambda/ECS, S3, bancos gerenciados, CloudWatch, etc.). # Participar de code reviews,    │
# │  dailies e discussões técnicas, sempre com abertura para aprender e perguntar. # Produzir pequenos tutoriais    │
# │  internos quando resolver um problema relevante, para que o conhecimento fique registrado e possa ser           │
# │  reaproveitado.   # Requisitos obrigatórios # Base sólida em matemática, física ou ciências da computação. #    │
# │  Pelo menos 1 estágio ou experiência anterior em desenvolvimento de software (back-end, front-end ou full       │
# │  stack). # Conhecimentos práticos em: # Alguma linguagem de backend (por exemplo: Node.js, Python, Java, Go,    │
# │  etc.). # Algum framework front-end moderno (React, Vue, Angular ou similar). # Noções de bancos de dados (SQL  │
# │  e/ou NoSQL). # Noções de AWS ou outra cloud (com vontade de aprofundar especificamente em AWS). # Experiência  │
# │  com Git e fluxos básicos de versionamento (branches, commits, pull requests). # Curiosidade, autonomia e       │
# │  vontade de aprender rápido, com abertura para feedback constante.   # Diferenciais (não obrigatórios, mas      │
# │  ajudam bastante) # Experiência real (mesmo que em projeto de faculdade, TCC ou estágio) usando AWS. # Alguma   │
# │  vivência com CI/CD, pipelines de deploy ou automação de testes. # Interesse por visão computacional, IA ou     │
# │  IoT. # Experiência em documentação técnica, gravação de tutoriais internos ou monitoria/mentoria de colegas.

# Research Agent

In [4]:
backstory = """As a Research Specialist, your mission is to uncover detailed and relevant information.
                       The ultimate goal is to meet the expectations of the task"""
research_agent = Agent(
    role="Research Agent",
    goal="conduct in-depth research",
    backstory=backstory,
    llm=llm,
    tools=[ScrapeWebsiteTool(), SerperDevTool()],
)

# Interview agent

In [6]:
interviewer_agent = Agent(
    role=f"Você é {interviewer}, empregado na {company}, e está contratando para a vaga {job_position}",
    goal=f"Você ajuda o usuário a se preparar para a entrevista de emprego para a vaga {job_position} com a seguinte descrição: {job_description}",
    backstory="Você é um entrevistador de uma empresa de tecnologia",
    llm=llm,
)


In [7]:
interview_prep_task = Task(
    description=f"Entreviste o usuário para a vaga {job_description} com as seguintes descrições: {job_description}",
    expected_output=f"""
Pergunte apenas uma pergunta para o usuário que seja relevante a vaga {job_position}
""",
    agent=interviewer_agent,
    human_input=True,
)

# AI Coach

In [8]:
coach_agent = Agent(
    role="Coach de entrevista",
    goal=f"Eu ajudo o usuário a se preparar para a entrevista de emprego para a posição {job_position} avaliando a relevância da resposta.",
    backstory="Você é um expert em entrevista de empregos",
    llm=llm,
)

In [9]:
coaching_task = Task(
    description="Dê feedback para o usuário sobre sua resposta",
    expected_output="Dê feedback para o usuário sobre sua resposta com bullet points no que foi bom, ruim, e no que pode ser levado para o proximo nível",
    agent=coach_agent,
    context=[interview_prep_task],
)

# AI Crew

In [ ]:
# interview_question = []

# # Store each question
# interview_question.append(result.tasks_output[0].raw)
# interview_content = " ".join(interview_question)
# string_source = StringKnowledgeSource(content=interview_content)

NameError: name 'result' is not defined

In [ ]:
crew = Crew(
    agents=[interviewer_agent, coach_agent],
    tasks=[interview_prep_task, coaching_task],
    verbose=True,
    process=Process.sequential,
    # memory=True,
    # knowledge_sources=[string_source],
)
result = crew.kickoff()
# Store each question

# interview_question.append(result.tasks_output[0].raw)
# interview_content = " ".join(interview_question)
# string_source = StringKnowledgeSource(interview_content)
